In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import torchvision.utils as vutils

import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando {device} para entrenar')

## Cargar datos

In [ ]:
DATASET = os.path.abspath('real_and_fake_face/')
INITIAL_SIZE_OF_IMAGES = (128, 128)

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(INITIAL_SIZE_OF_IMAGES),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=DATASET, transform=data_transforms)

train_loader = torch.utils.data.DataLoader(dataset=dataset, shuffle=True)

In [ ]:
valid_size = int(0.8 * len(train_loader.dataset))
test_size = len(train_loader.dataset) - valid_size

valid_dataset, test_dataset = torch.utils.data.random_split(train_loader.dataset, [valid_size, test_size])

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False)

## Modelo

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64*32*32, 128)

        self.fc2 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64*32*32)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
model = CNN().to(device)
print(model)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(num_epochs):
    training_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        training_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = training_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_accuracy = 100 * correct_train / total_train
    train_accuracies.append(train_accuracy)

    model.eval()
    valid_loss = 0.0
    correct_valid = 0
    total_valid = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_valid += labels.size(0)
            correct_valid += (predicted == labels).sum().item()

    avg_valid_loss = valid_loss / len(valid_loader)
    valid_losses.append(avg_valid_loss)
    valid_accuracy = 100 * correct_valid / total_valid
    valid_accuracies.append(valid_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Valid Loss: {avg_valid_loss:.4f}, Valid Acc: {valid_accuracy:.2f}%')

    model.train()

print("Entrenamiento finalizado!")

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.legend()
plt.xlabel('Epochs')
plt.title('Loss vs. Epochs')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Valid Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.title('Accuracy vs. Epochs')

plt.show()

In [ ]:
model.eval()
with torch.no_grad(): # Deshabilita el cálculo de gradientes
    correct = 0
    total = 0
    for images, labels in test_loader:
        # pasar al dispositivo
        images = images.to(device)
        labels = labels.to(device)
        ############################
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) # Obtiene el índice de la clase con mayor probabilidad
        total += labels.size(0) # Tamaño del batch
        correct += (predicted == labels).sum().item() # Suma el número de predicciones correctas

    print(f'Precisión del modelo en las 10000 imágenes de prueba: {100 * correct / total}%')